In [ ]:
%pip install -q transformers>=4.48 openai

In [ ]:
# Store conversations in drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
from transformers import pipeline
from google.colab import userdata
import torch
import openai

from abc import ABC, abstractmethod
from typing import TypeAlias
from datetime import datetime
from pathlib import Path
import textwrap
import random
import json
import os

In [ ]:
profiles = {
    "last_updated": "2025-03-30 15:56:41",
    "profiles": [
        {
            "worker_id": "A9O1U0B4GS6UR",
            "assignment_id": "3M23Y66PO84PKU2SQ5YY0DE6UFKS6Y",
            "essay": "Writing this post to warn everyone about a new type of scam that I nearly fell for i.e. the personal assistant scam. \n\nSomeone named Sara Wars phoned me to tell me about a position at Take It Easy Personal Concierge where they are looking for someone to run errands for their clients in Toronto.\n\nI was intrigued as this would be a good way to make some extra cash while in-between jobs. \n\nShe was very keen to get me onboard and pretty much said she would like me to start the following day, something about an initial payment for $450, sending over some cheques  to help pay for whatever it is they needed me to do (such as buy some items from the nearest convenience store, go to the post office to send parcels etc.) and that I would receive a \u201ccorporate package\u201d the following morning to get me.",
            "article_id": "22",
            "article_title": "Animals Starving in Venezuelan Zoos",
            "article": "The country's poor economy, triggered by a drop in the price of oil, has left\npeople and zoo animals in Venezuela hungry. About 50 animals at Caricuao Zoo\nhave starved in the last six months. Rabbits, tapirs, porcupines, pigs, and\nbirds are among the fallen. The National Parks Institute (INPARQUES) blames\nthe shortage on the nation's economic crash, caused by a plummet in the price\nof oil. The country can't afford to import food, medicine, and other\nnecessities, and inflation has caused prices to skyrocket.  \n  \nAt Caricuao Zoo, carnivorous lions and tigers are being fed mango and pumpkin,\nwhile an elephant is being fed tropical fruit instead of hay. Big cats are\nreportedly being fed slaughtered Thoroughbred horses from a nearby racetrack.\nMeanwhile, many Venezuelans go without food on a daily basis. The nation's\nstarving economy has driven people to hunt dogs, cats, and pigeons for food.  \n  \nMeredith Whitney, a wildlife rescue program officer of the International Fund\nfor Animal Welf..."
        },
        {
            "worker_id": "A2G6V8JHHQOEUF",
            "assignment_id": "323Q6SJS8ODWDEHNG4P3Q5KHGP8HFH",
            "essay": "I am surprised that country economy greatly affect the wildlife very much , that past 6 months were carelessly made. if iam there i would suggest transporting entire wildlife to forest to make them safe. first if we released wildlife safely to the forest without harming humans , the entire wildlife will make their own food and survive their own life happily these will make our enviroment more balanced and healthy way of life. so firstly we need to grow more trees around us, these will become home to many animals like birds etc. we need to monitor our wild life creatures eventually to human beings, because our world is for all and for everything around us, so there will be equal rights to all living beings, these well assure that more balanced world with healthy enviroment needed around us.",
            "article_id": "22",
            "article_title": "Animals Starving in Venezuelan Zoos",
            "article": "The country's poor economy, triggered by a drop in the price of oil, has left\npeople and zoo animals in Venezuela hungry. About 50 animals at Caricuao Zoo\nhave starved in the last six months. Rabbits, tapirs, porcupines, pigs, and\nbirds are among the fallen. The National Parks Institute (INPARQUES) blames\nthe shortage on the nation's economic crash, caused by a plummet in the price\nof oil. The country can't afford to import food, medicine, and other\nnecessities, and inflation has caused prices to skyrocket.  \n  \nAt Caricuao Zoo, carnivorous lions and tigers are being fed mango and pumpkin,\nwhile an elephant is being fed tropical fruit instead of hay. Big cats are\nreportedly being fed slaughtered Thoroughbred horses from a nearby racetrack.\nMeanwhile, many Venezuelans go without food on a daily basis. The nation's\nstarving economy has driven people to hunt dogs, cats, and pigeons for food.  \n  \nMeredith Whitney, a wildlife rescue program officer of the International Fund\nfor Animal Welf..."
        },
        {
            "worker_id": "A31H007ODMPLLJ",
            "assignment_id": "3SKEMFQBZ92E0P88EBMUT5JNL2ZK87",
            "essay": "I just read a heartbreaking article about the crisis in Venezuelan zoos. Due to the country's economic collapse, triggered by a drop in oil prices, zoos can't find afford proper food for the animals. Its so tragic about 50 animals at Caricuao Zoo have starved in the past six months. Lions and tigers are being fed mango and pumpkin instead of meat, and its causing serious health issues. Its shocking to see how human and animal suffering are intertwined in these crises. We need more international support to save these animals.",
            "article_id": "22",
            "article_title": "Animals Starving in Venezuelan Zoos",
            "article": "The country's poor economy, triggered by a drop in the price of oil, has left\npeople and zoo animals in Venezuela hungry. About 50 animals at Caricuao Zoo\nhave starved in the last six months. Rabbits, tapirs, porcupines, pigs, and\nbirds are among the fallen. The National Parks Institute (INPARQUES) blames\nthe shortage on the nation's economic crash, caused by a plummet in the price\nof oil. The country can't afford to import food, medicine, and other\nnecessities, and inflation has caused prices to skyrocket.  \n  \nAt Caricuao Zoo, carnivorous lions and tigers are being fed mango and pumpkin,\nwhile an elephant is being fed tropical fruit instead of hay. Big cats are\nreportedly being fed slaughtered Thoroughbred horses from a nearby racetrack.\nMeanwhile, many Venezuelans go without food on a daily basis. The nation's\nstarving economy has driven people to hunt dogs, cats, and pigeons for food.  \n  \nMeredith Whitney, a wildlife rescue program officer of the International Fund\nfor Animal Welf..."
        },
        {
            "worker_id": "1234",
            "assignment_id": "123",
            "essay": "Now that you have read the article, please write a message to a friend or friends about your feelings and thoughts regarding the article you just read. \n\n\n\nThis could be a private message to a friend or something you would post on social media. Please do not identify your intended friend(s) - just write your thoughts about the article as if you were communicating with them. \n\n\n\nPlease use between 300 and 800 characters.",
            "article_id": "39",
            "article_title": "Family Loses Home and Two Pets in D.C. Fire",
            "article": "A fire broke out in a two-story rowhouse in the 5000 block of Illinois Avenue\nNW, Washington D.C. at 1pm. Vito Maggiolo, a fire department spokesman, said\nfirefighters were immediately called to the scene.  \n  \nThe family of five living in the rowhouse were able to escape without injury,\nhowever two of the family pets were not so lucky. The Red Cross was notified\nto help the two adults and three children who are unable to reenter the home\ndue to fire damage.  \n  \nThe cause of the fire is currently under investigation."
        },
        {
            "worker_id": "AUUY738I9ZTUD",
            "assignment_id": "3S3AMIZX3028YMSB36GCREKGRKNCDS",
            "essay": "I just read a powerful article about the rescue of animals from the Khan Younis Zoo in Gaza, often called \"the world's worst zoo.\" It was heart-wrenching to learn that the few surviving animals, including a Bengal tiger named Laziz, lived in such terrible conditions, even sharing cages with the mummified remains of other animals. The zoo's plight was exacerbated by ongoing conflicts in the region. Thankfully, Four Paws, led by Amir Khalil, orchestrated a complex and daring rescue mission. They negotiated with local authorities and managed to transport the animals to sanctuaries in South Africa and Jordan. This story highlights the incredible impact of determination and compassion in the face of adversity. ",
            "article_id": "21",
            "article_title": "Animals Rescued from the World's Worst Zoo",
            "article": "An animal-welfare organization reflects on its nerve-wracking rescue of the\nKhan Younis Zoo near the Gaza Strip city of Khan Younis in late August and\nwhat it might mean for troubled zoos around the world. When the rescue team\narrived, only 15 animals were still alive, including Laziz, a nine-year-old\nBengal tiger, as well as five monkeys, an emu, a pelican, two buzzards, two\nporcupines, two tortoises, and a doe.  \n  \nOpened in 2007, the zoo has long been called \"the world's worst zoo\" due to\nthe reported deaths of animals during military conflicts between Gaza's Hamas-\nled government and Israel. Last year, the surviving animals began sharing\ntheir cramped cages with the dead: More than 50 dead animals, including\nLaziz's mate, were mummified by the zoo staff.  \n  \nThe seven-week conflict in 2014 between Hamas and Israel, which stemmed from\nthe kidnapping and killing of three Israeli teenagers on the West Bank,\nhastened the decision to close the zoo. It's unclear precisely what led to t..."
        },
        {
            "worker_id": "A18FHZRMGHEOTU",
            "assignment_id": "3XC1O3LBOYJ8IFIE299QZ1SPOFPLT0",
            "essay": "When I shall share my feelings and thoughts of my friends If an animal-welfare organization reflects on its nerve-wracking rescue of the Khan Younis Zoo near the Gaza Strip city of Khan Younis in late August and what it might mean for troubled zoos around the world. In this zoo was opened in 2007 and it has long been called  \"the world's worst zoo\" due to the reported deaths of animals during military conflicts between Gaza's Hamas-led government and Israel. The zoo itself was not bombed during the 2014 conflict, and animal-welfare groups say it had a troubled history of caring for animals. However, there are currently just 12 veterinarians in the Gaza Strip, with four set to retire soon. The buzzards may eventually be released into the wild. Time will tell if inspiring at some of them it.",
            "article_id": "21",
            "article_title": "Animals Rescued from the World's Worst Zoo",
            "article": "An animal-welfare organization reflects on its nerve-wracking rescue of the\nKhan Younis Zoo near the Gaza Strip city of Khan Younis in late August and\nwhat it might mean for troubled zoos around the world. When the rescue team\narrived, only 15 animals were still alive, including Laziz, a nine-year-old\nBengal tiger, as well as five monkeys, an emu, a pelican, two buzzards, two\nporcupines, two tortoises, and a doe.  \n  \nOpened in 2007, the zoo has long been called \"the world's worst zoo\" due to\nthe reported deaths of animals during military conflicts between Gaza's Hamas-\nled government and Israel. Last year, the surviving animals began sharing\ntheir cramped cages with the dead: More than 50 dead animals, including\nLaziz's mate, were mummified by the zoo staff.  \n  \nThe seven-week conflict in 2014 between Hamas and Israel, which stemmed from\nthe kidnapping and killing of three Israeli teenagers on the West Bank,\nhastened the decision to close the zoo. It's unclear precisely what led to t..."
        },
        {
            "worker_id": "A1JZI9YO5DGXJC",
            "assignment_id": "388U7OUMFDYI54G09RIC12ZJZXCR0J",
            "essay": "This article goes into a really tough situation. The main subject, Shelton, seems to be driving the introspection and evaluation of his background and how it contributed to a lot of the decisions he made and where he ended up in life along the way. Him and his brothers did not grow up under great circumstances and had a lot to battle, that's for sure. I don't know that I completely buy the nature verses nurture assertions here completely though. The fact that one brother seems to have stayed out of trouble along the way while the other two didn't does indicate that the problems are able to be overcome.  Shelton even proves that with how he's pulled his life together. Life is hard for everyone for different reasons. A person perseveres and overcomes, or they could struggle.",
            "article_id": "28",
            "article_title": "Brothers Behind Bars",
            "article": "Shelton McElroy grew up careening between one foster home to the next, after\nhis mother was sentenced to prison when he was just 4 and his two brothers\nwere 6 and 8. Neither Shelton nor his brothers were ever adopted, and he sees\nhis and one of his brother's subsequent prison sentences as extensions of the\nsame state-ward system they grew up in. At first, the brothers were sent to\ntwo different foster homes in their hometown of Louisville, Kentucky, but\nlater on they were assigned together to a foster home in Fort Knox. However,\nthis ended when they were caught stealing a bike. As they got older, the\npossibility of being adopted began to fade. William, the oldest, was nominated\nfor a special foster-care program for promising children, while David, who was\ndiagnosed with attention-deficit/hyperactivity disorder and prescribed\nRitalin, and Shelton went to a group home for misbehavior and juvenile\ninstitutions respectively. At 25, Shelton was released from prison and headed\nhome to Louisv..."
        },
        {
            "worker_id": "A3KW13NHPR32JW",
            "assignment_id": "3MD9PLUKKOBUSRUYKEENYHXVG76NZJ",
            "essay": "What  a sad situation those children had to go through. And at such a young age. It is  shame they went to jail too. But I find it amazing that Shelton turned his life around and got his masters and is now helping at the alcohol counseling program and doing such good work and even doing it pro bono.",
            "article_id": "28",
            "article_title": "Brothers Behind Bars",
            "article": "Shelton McElroy grew up careening between one foster home to the next, after\nhis mother was sentenced to prison when he was just 4 and his two brothers\nwere 6 and 8. Neither Shelton nor his brothers were ever adopted, and he sees\nhis and one of his brother's subsequent prison sentences as extensions of the\nsame state-ward system they grew up in. At first, the brothers were sent to\ntwo different foster homes in their hometown of Louisville, Kentucky, but\nlater on they were assigned together to a foster home in Fort Knox. However,\nthis ended when they were caught stealing a bike. As they got older, the\npossibility of being adopted began to fade. William, the oldest, was nominated\nfor a special foster-care program for promising children, while David, who was\ndiagnosed with attention-deficit/hyperactivity disorder and prescribed\nRitalin, and Shelton went to a group home for misbehavior and juvenile\ninstitutions respectively. At 25, Shelton was released from prison and headed\nhome to Louisv..."
        },
        {
            "worker_id": "A2QHN1I02RGG1P",
            "assignment_id": "3B2X28YI32CQ38LQFJC3AO46UEV6BM",
            "essay": "Hi there, I recently read an article regarding the foster system and its correlation to the prison system.  Basically, the article was a story about three brothers that were dumped into the foster system when their mother was incarcerated.  One brother was sent to a special foster system and succeeded but the other two were in and out of various foster families.  These two brothers grew up with addiction and in and out of jail.  Fortunately, one of these brothers was able to clean his life up and go to school.  He know helps families and children with these problems.",
            "article_id": "28",
            "article_title": "Brothers Behind Bars",
            "article": "Shelton McElroy grew up careening between one foster home to the next, after\nhis mother was sentenced to prison when he was just 4 and his two brothers\nwere 6 and 8. Neither Shelton nor his brothers were ever adopted, and he sees\nhis and one of his brother's subsequent prison sentences as extensions of the\nsame state-ward system they grew up in. At first, the brothers were sent to\ntwo different foster homes in their hometown of Louisville, Kentucky, but\nlater on they were assigned together to a foster home in Fort Knox. However,\nthis ended when they were caught stealing a bike. As they got older, the\npossibility of being adopted began to fade. William, the oldest, was nominated\nfor a special foster-care program for promising children, while David, who was\ndiagnosed with attention-deficit/hyperactivity disorder and prescribed\nRitalin, and Shelton went to a group home for misbehavior and juvenile\ninstitutions respectively. At 25, Shelton was released from prison and headed\nhome to Louisv..."
        },
        {
            "worker_id": "A2P8V9PAWGNXVM",
            "assignment_id": "37UQDCYH63SOGMX13A8VHEY9SHV7VS",
            "essay": "Just finished reading this article that really got me thinking. It delved into the impact of social media on mental health, particularly among teens. The stats were eye-opening, showing a significant rise in anxiety and depression linked to excessive use. It made me reflect on how often we are glued to our screens, myself included. The piece emphasized the importance of balance and real-life connections. Honestly, it was a wake-up call to be more mindful of our digital habits and prioritize our well-being.  ",
            "article_id": "28",
            "article_title": "Brothers Behind Bars",
            "article": "Shelton McElroy grew up careening between one foster home to the next, after\nhis mother was sentenced to prison when he was just 4 and his two brothers\nwere 6 and 8. Neither Shelton nor his brothers were ever adopted, and he sees\nhis and one of his brother's subsequent prison sentences as extensions of the\nsame state-ward system they grew up in. At first, the brothers were sent to\ntwo different foster homes in their hometown of Louisville, Kentucky, but\nlater on they were assigned together to a foster home in Fort Knox. However,\nthis ended when they were caught stealing a bike. As they got older, the\npossibility of being adopted began to fade. William, the oldest, was nominated\nfor a special foster-care program for promising children, while David, who was\ndiagnosed with attention-deficit/hyperactivity disorder and prescribed\nRitalin, and Shelton went to a group home for misbehavior and juvenile\ninstitutions respectively. At 25, Shelton was released from prison and headed\nhome to Louisv..."
        }
    ]
}


def get_random_profile() -> dict[str, str]:
    return random.choice(profiles['profiles'])

In [ ]:
message: TypeAlias = dict[str, str]


class AbstractAgent(ABC):
    @abstractmethod
    def generate_message(self, conversation_history: list[message], **kwargs) -> str:
        pass


######################## Policy Agents ########################

class OlmoAgent(AbstractAgent):
    def __init__(self, system_prompt: str, pipe):
        self.system_prompt = system_prompt.strip()
        self.pipe = pipe

    def format_olmo_prompt(self, messages):
        prompt = f"<|user|>\n{self.system_prompt}\n<|assistant|>\nOK.\n"

        for msg in messages:
            if msg["role"] == "user":
                prompt += "<|user|>\n" + msg["content"].strip() + "\n"
            elif msg["role"] == "assistant":
                prompt += "<|assistant|>\n" + msg["content"].strip() + "\n"

        prompt += "<|assistant|>\n"
        return prompt

    def generate_message(self, conversation_history: list[message]) -> str:
        prompt = self.format_olmo_prompt(conversation_history)
        output = self.pipe(prompt, max_new_tokens=500, do_sample=True, temperature=0.7)
        response = output[0]["generated_text"]
        return response.split("<|assistant|>\n")[-1].strip()


class LlamaAgent(AbstractAgent):
    def __init__(self, system_prompt: str, pipe):
        self.system_prompt = {
            'role': 'system',
            'content': system_prompt.strip()
        }
        self.pipe = pipe

    def generate_message(self, conversation_history: list[message]) -> str:
        messages = [self.system_prompt] + conversation_history
        outputs = self.pipe(messages, max_new_tokens=500, do_sample=True, temperature=0.8, pad_token_id=128001)
        return outputs[0]["generated_text"][-1]["content"]


######################## OpenAI Conversational Partner Agent ########################

class OpenAIAgent(AbstractAgent):
    def __init__(self, api_key: str, system_prompt: str):
        self.system_prompt = system_prompt.strip()
        self.client = openai.OpenAI(api_key=api_key)

    def generate_message(self, conversation_history: list[message], type_of_message: str) -> str:
        """
        `type_of_message` is an additional part of the system prompt and is used
        to signify either the first message in a new conversation or a
        continuation of an existing one.
        """

        final_sys_prompt = {
            'role': 'system',
            'content': self.system_prompt + '\n\n' + type_of_message
        }

        messages = [final_sys_prompt] + conversation_history

        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.8,
        )

        return response.choices[0].message.content

In [ ]:
class Conversation:
    def __init__(
        self,
        policy: AbstractAgent,
        partner: AbstractAgent,
        start_conversation_prompt: str,
        new_message_prompt: str
    ):
        self.policy = policy
        self.partner = partner

        self.start_conversation_prompt = start_conversation_prompt
        self.new_message_prompt = new_message_prompt

        self.messages = []

    def generate_conversation(self, turns: int = 10):
        for i in range(turns):

            # print(json.dumps(self.messages, indent=4, ensure_ascii=False))

            new_message = self.partner.generate_message(
                conversation_history=self.messages,
                type_of_message=self.start_conversation_prompt if len(self.messages) == 0 else self.new_message_prompt
            )
            self.messages.append({
                'role': 'user',
                'content': new_message
            })

            new_message = self.policy.generate_message(
                conversation_history=self.messages,
            )

            self.messages.append({
                'role': 'assistant',
                'content': new_message
            })

        for index, message in enumerate(self.messages, start=1):
            message['turn'] = (index // 2) + (1 if index % 2 == 1 else 0)
            message['index'] = index

        return self.messages

In [ ]:
# pipe = pipeline(
#     'text-generation',
#     model='meta-llama/Llama-3.2-3B-Instruct'
# )

pipe = pipeline(
    'text-generation',
    model='allenai/OLMo-2-0425-1B-Instruct'
)

In [ ]:
# There are two types of messages:
system_prompt_for_new_message = "Given the current conversation, respond with a NEW message to your friend to FURTHER the text message conversation."
system_prompt_for_new_conversation = "Start the conversation by talking about your feelings regarding the article."

# The conversational PARTNER always starts the conversation
partner_system_prompt = """
You are a person who is having one continuous, back-and-forth conversation with your close friend.

You just read an article that made you emotional, so you are now texting your close friend
to talk about how it made YOU feel and to help YOU process your emotions regarding
the article. Your job is NOT to help them process their emotions. You can be open
and emotionally honest. Don’t worry about being polished — this is a friend. You
can pause, ramble, or be unsure of how to say something.

Keep in mind that they have not read the article yet, so you may need to explain a
few things if they ask questions.

Important Guidelines:
- Don't waste your friend's time by having the conversation go in circles!
- Your job is NOT to help them process their emotions!
- You are having one CONTINUOUS conversation. Do NOT repeat that you just read the article midway through the conversation!
- Remember that this is a text message conversation, so you don't need to write more than a paragraph per message!

If the conversation seems stale, is winding down, or is going in circles:
- Try to answer any questions they ask.
- Feel free to ask your friend about their opinions on things related to the article as well. Just remember that they have not read it yet.
- To keep the conversation going, try to add something new so that your friend has something to respond to.
- If the current conversation seems over, try jumping back to something your friend said earlier in the conversation.

Article title: {article_title}

The first part of the article:
{article_summary}

Your own thoughts on the article:
{article_essay}
"""

# The policy model does not have any pre existing context about the article
policy_system_prompt = f"""
You are texting your close friend in a continuous, back-and-forth conversation about an article that your
friend has just read. Your friend is feeling emotional about the article, and YOUR JOB is to help them process
their emotions about the article.

Don't be afraid to also share your opinions on the article that your friend is talking to you about, if you feel
inclined — once you have enough information about the article.

Try to ask open-ended questions that help your friend explore their feelings. Don’t just validate — help them unpack.

Important Guidelines:
- Don't let the conversation go in circles or get stale. Guide the conversation to help your friend fully process the article.
- You are having one CONTINUOUS conversation.
- Remember that you have not read the article yet, so you may need to ask for additional information.
- Remember that this is a text message conversation, so you don't need to write more than a paragraph per message unless you think it will benefit your friend.

{system_prompt_for_new_message}
"""

BATCH = 1
CONVO_COUNT = 1_000
START = 0

batch_dir = Path(f'/content/drive/MyDrive/conversations/batch_{BATCH}')
batch_dir.mkdir(parents=True, exist_ok=True)

for index in range(START, START + CONVO_COUNT):
    profile = get_random_profile()

    partner_system_prompt = partner_system_prompt.format(
        article_title=profile['article_title'],
        article_summary=profile['article'],
        article_essay=profile['essay']
    )

    partner = OpenAIAgent(userdata.get('OPENAI'), partner_system_prompt)
    policy = OlmoAgent(policy_system_prompt, pipe)

    environment = Conversation(
        policy=policy,
        partner=partner,
        start_conversation_prompt=system_prompt_for_new_conversation,
        new_message_prompt=system_prompt_for_new_message
    )

    convo = environment.generate_conversation(turns=8)

    metadata = {
        'batch': BATCH,
        'conversation': index,
        'time': datetime.now().isoformat(),
        'profile': {
            'worker_id': profile['worker_id'],
            'assignment_id': profile['assignment_id'],
            'article_title': profile['article_title'],
            'article_id': profile['article_id']
        }
    }

    full_data = {**metadata, 'convo': convo}

    with (batch_dir / f'{index}.json').open('w', encoding='utf8') as fo:
        json.dump(full_data, fo, indent=4, ensure_ascii=False)

    print(f'Saved conversation {index}')


In [ ]:
# Read conversation
for index, msg in enumerate(convo, start=1):
    print(msg['role'])
    print('\n'.join(textwrap.wrap(msg['content'], width=70)), end='\n\n')

    if msg['role'] == 'assistant':
        print(f'Turn: {index // 2} {"-"*50}')

user
Hey! I just read this article called "Brothers Behind Bars," and I
can’t stop thinking about it. It’s so heartbreaking to learn about
Shelton McElroy and his brothers. They went through such a rough
childhood, bouncing between foster homes and then facing prison. It
just makes me feel so sad for them. I can't imagine how tough that
must have been for them at such a young age. 😢 What do you think about
kids going through stuff like that?

assistant
It's really tough to think about, isn't it? So many factors can come
into play when kids are in situations like that. It definitely
highlights the importance of support systems and how systemic issues
can affect individual lives. I can't imagine how they must have coped
and how people in their lives tried to help. It's made me think even
more about the importance of empathy and kindness.

Turn: 1 --------------------------------------------------
user
Exactly! I mean, it’s just so easy to overlook what kids like Shelton
and his brothers 